Welcome to the Julia tutorial 👋! 

We're largely going to be comparing Julia and Python 🐍 across two dimensions: syntax and speed. 

We will also explore one of those fundamental julia concepts: multiple dispatch.

## 0. Load packages

We're going to load [BenchmarkTools](https://github.com/JuliaCI/BenchmarkTools.jl), a nice package that performance tracks Julia code and [PyCall](https://github.com/JuliaPy/PyCall.jl), a Julia package that lets you call python functions from the Julia language.

In [1]:
using Pkg
Pkg.add("BenchmarkTools")
Pkg.add("PyCall")

using BenchmarkTools 
using PyCall

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed FFMPEG_jll ─ v4.4.2+2
  No Changes to `~/.julia/environments/v1.7/Project.toml`
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [b22a6f82] ↑ FFMPEG_jll v4.4.2+1 ⇒ v4.4.2+2
Precompiling project...
  ✓ FFMPEG_jll
  ✗ FFMPEG
  ✗ GR_jll
  1 dependency successfully precompiled in 6 seconds (171 already precompiled)
  2 dependencies errored. To see a full report either run `import Pkg; Pkg.precompile()` or load the packages
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


## Syntax

First up, the Julia syntax. 

We're going to cover the bare minimum of how to print, assign variables, comment, some basic syntax for math and some string operations in Julia.

### 0. How to print

In [2]:
println("I'm excited to learn Julia!")

I'm excited to learn Julia!


### 1. Assign variables

In [3]:
my_answer = 42
typeof(my_answer)

Int64

In [4]:
my_pi = 3.14159
typeof(my_pi)

Float64

you can assign to emojis in julia...

In [5]:
😺 = "smiley cat!"
typeof(😺)

String

In [6]:
😀 = 3
😀

3

In [7]:
😞 = -1
😞

-1

In [8]:
emoji_math = 😀 + 😞

2

In [9]:
println(emoji_math)

2


you can comment single lines in the same way as you do in Python. For lines of code, you can:

In [10]:
#=

For multi-line comments, 
use the '#= =#' sequence.

=#

The basic syntax to add, subtract divide and multiple is the same in Julia. 

Can you assign 10 to one variable and 20 to another variable and add them together to create a new variable? Can you convert the new variable to a float?

As for strings, much like how you use f' in python, you can use the $ sign to insert existing variables into a string.

In [11]:
name = "Jane"
num_fingers = 10
num_toes = 10

10

In [12]:
println("Hello, my name is $name.")
println("I have $num_fingers fingers and $num_toes toes.")

Hello, my name is Jane.
I have 10 fingers and 10 toes.


In [13]:
println("That is $(num_fingers + num_toes) digits in all!!")

That is 20 digits in all!!


Can you complete the following exercise?

Declare two variables

```julia
a = 3
b = 4
```
and use them to create two strings:
```julia
"3 + 4"
"7" 
```
and store the results in `c` and `d` respectively

## Speed

We're now going to show off Julia's speed by comparing a very simple operation in Julia and in Python.

Consider the sum function sum(a), which sums all the elements of a.

In [14]:
a = rand(10^7) # 1D vector of random numbers, uniform

10000000-element Vector{Float64}:
 0.7524167785275826
 0.8092798746574756
 0.8483658058913708
 0.8028556284283226
 0.9780116405801058
 0.5107331040023988
 0.05231557612799609
 0.06242203692598414
 0.7523835858893788
 0.17009890172762598
 0.1170748898530064
 0.487256085186065
 0.3906302330651732
 ⋮
 0.5596092740894731
 0.018938277498184664
 0.3733819087957816
 0.6027001577479931
 0.7749348881360296
 0.5471964908401885
 0.7424165852294232
 0.37471839611327884
 0.16271799379051566
 0.38677756772407923
 0.8621695645216663
 0.5456069187723294

In [15]:
sum(a)

d = Dict()  # a "dictionary", i.e. an associative array
j_bench = @benchmark sum(a)
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any, Any} with 1 entry:
  "Julia built-in" => 1.83708

We can see how long this same operation will take in Python using PyCall, a package that provides a Julia interface to Python. 

In [16]:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [17]:
pysum(a)

4.998709925567024e6

In [18]:
pysum(a) ≈ sum(a)

true

In [19]:
py_list_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  809.914 ms … 818.890 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     816.004 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   815.285 ms ±   3.082 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                █                      ███         █       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█ ▁
  810 ms           Histogram: frequency by time          819 ms <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [20]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any, Any} with 2 entries:
  "Julia built-in"  => 1.83708
  "Python built-in" => 809.914

What about in hand-written Julia?

In [21]:
function mysum(A)   
    s = 0.0 # s = zero(eltype(a))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [22]:
j_bench_hand = @benchmark mysum($a)
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6

9.382

How do these the three methods compare? Reflect on these benchmark scores:

In [23]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia built-in..............1.8
Julia hand-written..........9.4
Python built-in...........809.9


meanwhile, to also explore the role defining types has in Julia's speed, consider the two following dictionaries, d1 and d2. 

We don't define any types in dictionary d1 but do in d2. d2 is a dictionary with string keys and array elements as values. We will pass these dictionaries into the function f1. 


In [24]:
d1 = Dict()
d2 = Dict{Char, Int}()

function f1(d, x::Int)
    d[convert(Char, x)] = x
end

f1 (generic function with 1 method)

lets benchmark this function with d1, the dictionary without any types declared.

In [25]:
@benchmark for i = 1:50
    f1(d1, i)
       end

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.838 μs …  4.514 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.856 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.875 μs ± 99.440 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃▆█▆▆▂                         ▁▁▂                         ▂
  ███████▃▄▄▅▄▅▄▅▄▇▆▆▆▆▄▃▅▆▅▅▄▄▅▅█████▅▆▅▅▄▁▃▄▁▆▄▄▄▄▃▄▃▄▁▄▄▃ █
  2.84 μs      Histogram: log(frequency) by time     3.24 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

lets do the same again with d2, the dictionary with declared types.

In [26]:
@benchmark for i = 1:50
    f1(d2, i)
       end

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.308 μs …  12.450 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.350 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.363 μs ± 142.619 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   █   ▇                                                       
  ▃█▆▂▅█▇▄▄▄▃▃▃▂▃▃▂▂▂▃▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▁▂▁▂▂▂▁▂▂▁▁▂▂▂▁▂▂▂▂▂▁▂▂▂ ▃
  1.31 μs         Histogram: frequency by time        1.75 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

see? faster still! ⏩ 

## Multiple Dispatch

remember: Multiple dispatch refers to when a function behaves differently depending on the types of its arguments. Let's follow an example of basic dispatch.

### 0. Basic Dispatch

One example of basic dispatch is demonstrated with the function f below: 

In [27]:
f(a, b) = "fallback"
f(a::Number, b::Number) = "a and b are both numbers"
f(a::Number, b) = "a is a number"
f(a, b::Number) = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 5 methods)

In [28]:
methods(f)

# 5 methods for generic function "f":
[1] f(a::Integer, b::Integer) in Main at In[27]:5
[2] f(a::Number, b::Number) in Main at In[27]:2
[3] f(a::Number, b) in Main at In[27]:3
[4] f(a, b::Number) in Main at In[27]:4
[5] f(a, b) in Main at In[27]:1

In [29]:
f(1.5, 2)

"a and b are both numbers"

In [30]:
f(1, "bar")

"a is a number"

In [31]:
f(1, 2)

"a and b are both integers"

In [32]:
f("foo", [1,2])

"fallback"

Can you write a function that repeats a string an integer number of times which takes the arguments in either order?


**hints:** you declare a type with `::`, you can use `repeat` to repeat elements n times. 

As a reminder, a typical function in julia looks like this:

```
function f(x,y)
           x + y
       end
```

where you 'end' a function, rather than 'return'.
       
or, the compact "assignment form" of the above function:

```
f(x,y) = x + y
```

both work! 

## Julia free for alls 

If you've finished up the exercises in this notebook, feel free to just play around with the language and write a few lines of Julia code! 

Perhaps try out using one of the libraries mentioned in the [Julia TL;DR](https://github.com/nestauk/ds_meetings/tree/main/tutorials/julia#libraries). 